In [ ]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf
import os
import joblib
import re
import os
import numpy as np
from src.training import gpt_trainer

In [ ]:
model = TFGPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

**Path Constants**

In [60]:
DATA_PATH = 'resources/gpt/data'
ORDERS_PATH = os.path.join(DATA_PATH, 'orders.txt')
ENQUIRY_PATH = os.path.join(DATA_PATH, 'enquiry.txt')
COMPLAINS_PATH = os.path.join(DATA_PATH, 'complains.txt')

**Convert into Sequences**

In [61]:
final_sequence_order = gpt_trainer.file_to_sequences(ORDERS_PATH, intent='order')
final_sequence_enquiry = gpt_trainer.file_to_sequences(ENQUIRY_PATH, intent='enquiry')
final_sequence_complain = gpt_trainer.file_to_sequences(COMPLAINS_PATH, intent='complain')

In [64]:
final_sequences = final_sequence_complain + final_sequence_enquiry + final_sequence_order

**Tokenize**

In [68]:
tokenizer.pad_token = tokenizer.eos_token
tokenized_inputs = tokenizer(final_sequences, truncation=True, padding=True, return_tensors="tf")

In [69]:
gpt_trainer.save_file('resources/gpt/data/tokenized_inputs.pkl', tokenized_inputs)